# Air Traffic Model Prototyping

This notebook defines a prototype air traffic model.

This model makes the following assumptions:
- A crew is available when each aircraft wants to depart (i.e. crew factors do not affect schedule),
- Each airport has one runway shared by arrivals and departures, and no limit to the number of aircraft who can be waiting to use it, so they are modeled as M/M/1/inf/FIFO queues

In [1]:
# Import and setup
import logging
import pprint

import pandas as pd
import matplotlib.pyplot as plt

import pyro

from bayes_air.model import air_traffic_network_model
from bayes_air.network import NetworkState
from bayes_air.schedule import parse_schedule

pyro.enable_validation(True)
pyro.set_rng_seed(1)
logging.basicConfig(level=logging.DEBUG)

# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

/home/cbd/.cache/pypoetry/virtualenvs/bayes-air-kfmWatce-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [4]:
# Define a simple schedule that sends the aircraft from airport 0 to airport 1 and back
# Time is defined in hours from the start of the simulation
schedule = pd.DataFrame(
    {
        "flight_number": ["F1", "F1"],
        "origin_airport": ["A1", "A2"],
        "destination_airport": ["A2", "A1"],
        "scheduled_departure_time": [0.0, 1.5],
        "scheduled_arrival_time": [1.0, 3.0],
        "actual_departure_time": [None, None],
        "actual_arrival_time": [None, None],
    }
)

flights, airports = parse_schedule(schedule)
airports[0].num_available_aircraft = 1
state = NetworkState(
    airports={airport.code: airport for airport in airports},
    pending_flights=flights,
)

In [5]:

pyro.render_model(
    air_traffic_network_model,
    model_args=(state, 5.0, 0.1),
    render_params=True,
    render_distributions=True,
)

[t=0.1] A1 has 1 aircraft available
[t=0.1] A2 has 0 aircraft available


RuntimeError: output with shape [] doesn't match the broadcast shape [0]
                       Trace Shapes:    
                        Param Sites:    
                       Sample Sites:    
        A1_mean_turnaround_time dist   |
                               value   |
        A2_mean_turnaround_time dist   |
                               value   |
           A1_mean_service_time dist   |
                               value   |
           A2_mean_service_time dist   |
                               value   |
              travel_time_A1_A2 dist   |
                               value   |
              travel_time_A2_A1 dist   |
                               value   |
F1_A1_A2_departure_service_time dist   |
                               value 0 |

In [7]:
simulated_state = air_traffic_network_model(state, 5.0, 0.1)
pprint.pprint(simulated_state)

Sampling service time 0.151164710521698 with rate 6.532366752624512
Sampling service time 0.024991368874907494 with rate 5.522851943969727
Sampling service time 0.17981043457984924 with rate 5.522851943969727
Sampling service time 0.11915569752454758 with rate 6.532366752624512
State(in_flight_aircraft=[],
      airports={'A1': Airport(code='A1',
                              runway_queue=[],
                              parked_aircraft=[(Aircraft(tail_number='N1',
                                                         itinerary=[ItineraryItem(origin='A1',
                                                                                  destination='A2',
                                                                                  scheduled_departure_time=0.0,
                                                                                  scheduled_arrival_time=1.0,
                                                                                  actual_departure_time=tensor(0

/home/cbd/src/mit/bayes_air/bayes_air/model.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(1.0 / airport_service_times[airport.code])
